In [45]:
import pandas as pd
from rake_nltk import Rake
import seaborn as sns
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

raw = pd.read_csv('professionalsKeywords.csv')
raw = raw[raw.columns[~raw.columns.str.contains('Unnamed:')]] #purge empty columns

#count = CountVectorizer()
#count_matrix = count.fit_transform(df['answered_questions'])

# generating the cosine similarity matrix
#cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [124]:
'''
User I/O (Olivia)

- should be able to enter a pro id chosen at random and print 10 recommended questions

- (will probably require work from Destiny's code)

'''
#choose randomly from list of valid professionals
def randomProfessional():
    global raw
    return raw.sample()

#This is where the math happens
#Um....will return a list of 10...dataframes?
#example output:
# id: 12345 question_title: How to turn shower on
#so yeah either a list of 10 dfs, or just 1 dataframe of this format:

#  question_ID  |   question_title                      | 
#--------------------------------------------------------
#     123...    |   How do I turn the shower on?        |
#     4f6...    |   I want to be a nurse. What's the... |
#     3cb...    |   Please help! (engineering)          |

def getRecommendations(professional_id):
    dummy = [['123','How do I turn the shower on?'],
             ['4f6','Some of yall is fat, and ugly, and unnattractive. but that is okay.'],
             ['d7c','Oh god! Im a psych major!']]
    
    qs = pd.DataFrame(columns=['question_id','question_title'], data=dummy)
    qs.set_index('question_id',inplace=True)
    return qs

def main():
    choice = input("Enter a professional's ID for 10 recommendations...or just press enter for a random one.")
    if choice is None or choice == '': #idk which one happens when you enter nothing
        choice = randomProfessional()
        print('the professional you got is ')
        print(choice['professionals_id'].iloc[0])
        print('with these keywords')
        print(choice['answered_questions'].iloc[0])
    try:
        recs = getRecommendations(choice)
        print('10 questions to recommend to this professional:')
        for q in recs.iterrows():
            #q will be a tuple of values (question_id, question_text)
            print(q[1][0])
            
            
    except Exception as e:
        print('had an oopsie...')
        print(e)
        
main()


Enter a professional's ID for 10 recommendations...or just press enter for a random one.
the professional you got is 
4358aa3c371a4f809fbff7b8f1a0c3a7
with these keywords
long college engineer go graduate future really considering becoming engineer architecture  
10 questions to recommend to this professional:
How do I turn the shower on?
Some of yall is fat, and ugly, and unnattractive. but that is okay.
Oh god! Im a psych major!


,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [126]:

#EXAMPLE!


df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

df = df[['Title','Genre','Director','Actors','Plot']]

df['Key_words'] = ""

dflist = np.array_split(df, 2)

profs = dflist[0]

quests = dflist[1]

#This creates the bag of words column and discards extra columns
#Tuned to movie data now, but should be easy enough to change into professionals data
def clean(data):
    for index, row in data.iterrows():
        plot = row['Plot']
        actors = row['Actors']
        director = row['Director']
        genre = row['Genre']
        # instantiating Rake, by default it uses english stopwords from NLTK
        # and discards all puntuation characters as well
        r = Rake()

        for item in [plot,actors,director,genre]:


            # extracting the words by passing the text
            r.extract_keywords_from_text(item)

            # getting the dictionary whith key words as keys and their scores as values
            key_words_dict_scores = r.get_word_degrees()

            # assigning the key words to the new column for the corresponding movie
            row['Key_words'] += ' '.join(list(key_words_dict_scores.keys()))


    # dropping the Plot column
    data.drop(columns = ['Plot'], inplace = True)
    data.drop(columns = ['Actors'], inplace = True)
    data.drop(columns = ['Director'], inplace = True)
    data.drop(columns = ['Genre'], inplace = True)
    data = data.set_index('Title')
    
clean(profs)
clean(quests)

count = CountVectorizer()
cm1 = count.fit_transform(profs['Key_words'])
cm2 = count.fit_transform(quests['Key_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(cm1, cm2)


indices = pd.Series(profs.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

recommendations('Baby Driver')

  (0, 411)	1
  (0, 443)	1
  (0, 1600)	1
  (0, 1869)	1
  (0, 813)	1
  (0, 201)	1
  (0, 716)	1
  (0, 1274)	1
  (0, 1630)	1
  (0, 462)	1
  (0, 355)	1
  (0, 2063)	1
  (0, 1547)	1
  (0, 612)	1
  (0, 35)	1
  (0, 1745)	1
  (0, 671)	1
  (0, 1348)	1
  (0, 203)	1
  (0, 1224)	1
  (0, 922)	1
  (0, 1923)	1
  (1, 391)	1
  (1, 691)	1
  (1, 1394)	1
  :	:
  (124, 51)	1
  (124, 274)	1
  (124, 1960)	1
  (124, 1842)	1
  (124, 863)	1
  (124, 1317)	1
  (124, 1990)	1
  (124, 1053)	1
  (124, 509)	1
  (124, 1054)	1
  (124, 1553)	1
  (124, 1880)	1
  (124, 794)	1
  (124, 8)	1
  (124, 1473)	1
  (124, 1493)	1
  (124, 1783)	1
  (124, 554)	1
  (124, 553)	1
  (124, 330)	1
  (124, 1908)	1
  (124, 1691)	1
  (124, 319)	1
  (124, 1966)	1
  (124, 1993)	1


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 2076 while Y.shape[1] == 2126